In [1]:
import pandas as pd
import numpy as np
import os

def generate_labels():
    print("⚙️ Cargando dataset crudo...")
    
    # --- RUTA CORREGIDA DE LECTURA ---
    input_path = "../data/nhanes_raw_data.csv"
    
    try:
        df = pd.read_csv(input_path)
    except FileNotFoundError:
        print(f"❌ Error: No encuentro '{input_path}'. Ejecuta el notebook 01 primero.")
        return

    initial_count = len(df)
    print(f"📄 Filas iniciales: {initial_count}")

    # 1. SANITY CHECKS
    mask_impossible = (
        (df['HGB'] > 25) | (df['HGB'] < 2) |
        (df['WBC'] > 200) | (df['WBC'] < 0.1) |
        (df['PLT'] > 2000) | (df['PLT'] < 0) |
        (df['Age'] > 120)
    )
    
    df_clean = df[~mask_impossible].copy()
    dropped = initial_count - len(df_clean)
    if dropped > 0:
        print(f"🧹 Sanity Check: Se eliminaron {dropped} filas con valores imposibles.")

    # 2. TARGET ENGINEERING
    cond_infection = df_clean['WBC'] > 11.0
    cond_thrombo = df_clean['PLT'] < 150.0
    cond_anemia = (
        ((df_clean['Sex'] == 1) & (df_clean['HGB'] < 13.0)) |
        ((df_clean['Sex'] == 2) & (df_clean['HGB'] < 12.0))
    )
    
    conditions = [cond_infection, cond_thrombo, cond_anemia]
    choices = ['Infection', 'Thrombocytopenia', 'Anemia']
    
    df_clean['Diagnosis'] = np.select(conditions, choices, default='Normal')
    
    # 3. REPORTE
    print("\n📊 Distribución de Clases Generada (Target):")
    print(df_clean['Diagnosis'].value_counts())
    print(f"\n% de Distribución:\n{df_clean['Diagnosis'].value_counts(normalize=True) * 100}")

    # --- RUTA CORREGIDA DE GUARDADO ---
    output_file = "../data/nhanes_labeled_data.csv"
    df_clean.to_csv(output_file, index=False)
    print(f"\n💾 Guardado exitosamente en: {output_file}")

if __name__ == "__main__":
    generate_labels()

⚙️ Cargando dataset crudo...
📄 Filas iniciales: 7528
🧹 Sanity Check: Se eliminaron 1 filas con valores imposibles.

📊 Distribución de Clases Generada (Target):
Diagnosis
Normal              5792
Anemia              1060
Infection            440
Thrombocytopenia     235
Name: count, dtype: int64

% de Distribución:
Diagnosis
Normal              76.949648
Anemia              14.082636
Infection            5.845622
Thrombocytopenia     3.122094
Name: proportion, dtype: float64

💾 Guardado exitosamente en: ../data/nhanes_labeled_data.csv
